In [25]:
# importer les librairies
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [27]:
search_query = input('Entrez le nom du produit :')

Entrez le nom du produit :stylos bic


In [28]:
# créer un agent de connexion
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept-Language': 'en-US, en;q=0.5'
}

search_query = search_query.replace(' ', '+')
base_url = 'https://www.amazon.com/s?k={0}'.format(search_query)

In [30]:
# disable your proxies in your requests
proxies = {
  "http": None,
  "https": None,
}


In [32]:
items = []
for i in range(1, 5):
    print('Processing {0}...'.format(base_url + '&page={0}'.format(i)))
    response = requests.get(base_url + '&page={0}'.format(i), headers=headers, proxies=proxies)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    results = soup.find_all('div', {'class': 's-result-item', 'data-component-type': 's-search-result'})

    for result in results:
        product_name = result.h2.text

        try:
            rating = result.find('i', {'class': 'a-icon'}).text
            rating_count = result.find_all('span', {'aria-label': True})[1].text
        except AttributeError:
            continue

        try:
            price1 = result.find('span', {'class': 'a-price-whole'}).text
            price2 = result.find('span', {'class': 'a-price-fraction'}).text
            price = float(price1 + price2)
            product_url = 'https://amazon.com' + result.h2.a['href']
            # print(rating_count, product_url)
            items.append([product_name, rating, rating_count, price, product_url])
        except AttributeError:
            continue
    sleep(1.5)

Processing https://www.amazon.com/s?k=stylos+bic&page=1...
Processing https://www.amazon.com/s?k=stylos+bic&page=2...
Processing https://www.amazon.com/s?k=stylos+bic&page=3...
Processing https://www.amazon.com/s?k=stylos+bic&page=4...


In [33]:
# créer le dataframe
df = pd.DataFrame(items, columns=['produit', 'note', 'nombre_de_notes', 'prix', 'lien produit'])

In [34]:
df.to_excel('{0}.xlsx'.format(search_query), index=False)